<a href="https://colab.research.google.com/github/TayJen/H_Tatarsan2022/blob/main/H_Tatarstan2022_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Импорт библиотек + необходимые установки

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Здесь необходимо вставить путь, по которому будут располагаться все файлы, включая данные, модель и аннотации.



In [ ]:
%cd /content/drive/Shareddrives/data_drive/H_Tatarstan2022
!pwd

Также в данной папке необходимо создать папку `data`. Внутри нее разместить папку `zipped`, в которой будут находиться используемые данные (`train_dataset_train.zip` и `test_dataset_test.zip`)

Таким образом полный путь изначальных данных будет:

*   `./data/zipped/train_dataset_train.zip`
*   `./data/zipped/test_dataset_test.zip`

In [ ]:
!unzip data/zipped/train_dataset_train.zip -d data/
!unzip data/zipped/test_dataset_test.zip -d data/

Устанавливаем необходимое для модели (модель - Yolov7)

In [ ]:
!git clone https://github.com/WongKinYiu/yolov7.git
!pip install --quiet -r yolov7/requirements.txt
!pip install --quiet map-boxes

In [ ]:
import os
import pandas as pd
import numpy as np

np.random.seed(59)

## Подготовка данных

In [ ]:
# Создадим обучающий датасет в формате yolo
!mkdir ./data_for_yolo
!mkdir ./data_for_yolo/data
!mkdir ./data_for_yolo/data/images
!mkdir ./data_for_yolo/data/labels
!mkdir ./data_for_yolo/data/images/train
!mkdir ./data_for_yolo/data/labels/train
!mkdir ./data_for_yolo/data/images/test
!mkdir ./data_for_yolo/data/labels/test

# Для создания аннотаций в нужном формате
!mkdir ./data/train/yolo_labels

In [ ]:
import yaml

# Последняя версия модели Yolo - Yolov7 немного привередлива и требует абсолютные пути
# до данных вместо относительных
def create_yaml_cont():
    """
        train: /content/drive/Shareddrives/data_drive/H_Tatarstan2022/data_for_yolo/data/images/train/
        val: /content/drive/Shareddrives/data_drive/H_Tatarstan2022/data_for_yolo/data/images/test/

        # number of classes
        nc: 5

        # class names
        names: ['human', 'head', 'face', 'car', 'carplate']
    """

    yaml_content = {
        'train': '/content/drive/Shareddrives/data_drive/H_Tatarstan2022/data_for_yolo/data/images/train/',
        'val': '/content/drive/Shareddrives/data_drive/H_Tatarstan2022/data_for_yolo/data/images/test',
        'nc': 5,
        'names': ['human', 'head', 'face', 'car', 'carplate']
    }

    with open('data_for_yolo/dataset.yaml', 'w') as f_yml:
        yaml.dump(yaml_content, f_yml)


create_yaml_cont()

In [ ]:
# Сделаем разметку как в Yolo, после этого нужным образом перемешаем и
# закинем в папку data_for_yolo
from tqdm import tqdm


name_to_class_num = {
    'human': 0,
    'head': 1,
    'face': 2,
    'car': 3,
    'carplate': 4
}

def yolo_annotation():
    old_path = './data/train/labels/'
    new_path = './data/train/yolo_labels/'

    for img_name in tqdm(os.listdir('data/train/images/')):
        img_name = img_name.split('.')[0]
        with open(new_path + img_name + '.txt', 'w') as f_new:
            for f in os.listdir(old_path):
                if f.startswith(img_name):
                    class_name = f.split('.')[0].split('_')[-1]
                    class_num = name_to_class_num[class_name]
                    with open(old_path + f, 'r') as f_old:
                        for row in f_old.readlines():
                            f_new.write(str(class_num) + row[1:])


yolo_annotation()

In [ ]:
from sklearn.model_selection import train_test_split


# Используем 20% датасета для валидации
images_names = os.listdir('./data/train/images')
train_images, test_images = train_test_split(images_names, test_size=0.2,
                                             shuffle=True, random_state=59)

len(train_images), len(test_images)

In [ ]:
train_images = set(train_images)
test_images = set(test_images)

len(train_images), len(test_images)

In [ ]:
from shutil import copy
from tqdm import tqdm


def copy_to_yolo():
    old_path_img = './data/train/images/'
    old_path_lbl = './data/train/yolo_labels/'
    yolo_path_img = './data_for_yolo/data/images/'
    yolo_path_lbl = './data_for_yolo/data/labels/'

    for image_name in tqdm(os.listdir(old_path_img)):
        lbl_name = image_name.split('.')[0] + '.txt'
        # print(image_name, lbl_name)
        if image_name in train_images:
            copy(old_path_img + image_name, yolo_path_img + 'train/' + image_name)
            copy(old_path_lbl + lbl_name, yolo_path_lbl + 'train/' + lbl_name)
        elif image_name in test_images:
            copy(old_path_img + image_name, yolo_path_img + 'test/' + image_name)
            copy(old_path_lbl + lbl_name, yolo_path_lbl + 'test/' + lbl_name)


copy_to_yolo()

In [ ]:
len(os.listdir('./data_for_yolo/data/images/train')), len(os.listdir('./data_for_yolo/data/images/test'))

In [ ]:
len(os.listdir('./data_for_yolo/data/labels/train')), len(os.listdir('./data_for_yolo/data/labels/test'))

## Модель

In [ ]:
!nvidia-smi

In [ ]:
path_to_data = './data_for_yolo/dataset.yaml'

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./runs/train

В результате многочисленных экспериментов лучший результат показала модель с разрешением 832 (`batch_size` равен 6, так как бесплатная GPU от Google Colab'a не вмещает в себя больше).

Также возможно лучшее разрешение показало бы лучшие результаты, но в бесплатной версии обучение идет слишком долго и память слишком мала.

!!! Внимание, чтобы обучение модели запустилось без ошибок, на этом этапе необходимо в созданную папку yolov7 загрузить веса, скачанные с официального репозитория модели: [YOLOv7](https://github.com/WongKinYiu/yolov7)

В разделе `Transfer Learning` имеются веса предобученных моделей различной сложности. В данном решении используются веса средней по сложности и памяти модели `yolov7x_training.pt`

После скачивания весов их необходимо поместить в папку модели, если этого не сделать то будет выдаваться ошибка об отсутствии возможности скачать веса

In [ ]:
!python ./yolov7/train.py --img 832 --batch 6 --epochs 100 --freeze 10 --data {path_to_data} --weights ./yolov7/yolov7x_training.pt --name yolov7x_general_832_results --cache

Модель обучается около 4-5 часов, если не следить за ней то она может вылететь

### Предикт для тестовой части

По окончанию обучения модели веса лучшей модели за все обучение будут сохранены в папке которая была указана при запуске обучения, в разделе `weights` под названием `best.pt`.

Лучший порог `--conf` оказался равным 0.01

In [ ]:
!python3 ./yolov7/detect.py --weights ./runs/train/yolov7x_general_832_results/weights/best.pt --img 832 --conf 0.01 --source ./data_for_yolo/data/images/test --save-txt --save-conf

Функции для получения меток и границ объектов (для теста еще и уверенности) из текстовых файлов в списки

In [ ]:
def get_soluton_labels(path_to_txt_folder):
    solutions = []
    for detection_file in os.listdir(path_to_txt_folder):
        img_name = detection_file.split('.')[0] + '.jpg'
        with open(path_to_txt_folder + detection_file, 'r') as f:
            data = f.read()
            data = [i for i in data.split('\n') if i != '']
        for line in data:
            val = [float(i) for i in line.split()]
            cls, xywh, conf = val[0], val[1:5], val[5]
            center_x, center_y, width, height = xywh
            xmin = center_x - (width / 2)
            xmax = center_x + (width / 2)
            ymin = center_y - (height / 2)
            ymax = center_y + (height / 2)
            solutions.append([img_name, int(cls), conf, xmin, xmax, ymin, ymax])

    return solutions

def get_test_labels(path_to_txt_folder):
    solutions = []
    for detection_file in os.listdir(path_to_txt_folder):
        img_name = detection_file.split('.')[0] + '.jpg'
        with open(path_to_txt_folder + detection_file, 'r') as f:
            data = f.read()
            data = [i for i in data.split('\n') if i != '']
        for line in data:
            val = [float(i) for i in line.split()]
            cls, center_x, center_y, width, height = val
            xmin = center_x - (width / 2)
            xmax = center_x + (width / 2)
            ymin = center_y - (height / 2)
            ymax = center_y + (height / 2)
            solutions.append([img_name, int(cls), xmin, xmax, ymin, ymax])
    return solutions

In [ ]:
solution_v7x = get_soluton_labels('./runs/detect/exp/labels/')
solution_v7x_df = pd.DataFrame(solution_v7x, columns=['ImageID', 'LabelName', 'Conf', 'XMin', 'XMax', 'YMin', 'YMax'])
solution_v7x_df.head()

In [ ]:
test_labels = get_test_labels('./data_for_yolo/data/labels/test/')
test_labels_df = pd.DataFrame(test_labels, columns=['ImageID', 'LabelName', 'XMin', 'XMax', 'YMin', 'YMax'])
test_labels_df.head()

Проверим скор на отложенной выборке

In [ ]:
from map_boxes import mean_average_precision_for_boxes as map_boxes_fn

mean_ap, average_precisions = map_boxes_fn(test_labels, solution_v7x_df,
                                           iou_threshold=0.25, verbose=False)
mean_ap

## Предсказание для теста

In [ ]:
!python3 ./yolov7/detect.py --weights ./runs/train/yolov7x_general_832_results/weights/best.pt --img 832 --conf 0.01 --source ./data/test/images --save-txt --save-conf

In [ ]:
solution_site_yolov7 = get_soluton_labels('./runs/detect/exp2/labels/')
solution_site_yolov7_df = pd.DataFrame(solution_site_yolov7, columns=['ImageID', 'LabelName', 'Conf', 'XMin', 'XMax', 'YMin', 'YMax'])
solution_site_yolov7_df.head()

Проверяем что нет никаких глупых ошибок

In [ ]:
solution_site_yolov7_df.describe()

При отправке решений необходимо соблюдать следующее соответствие классов:

`'car': 0, 'head': 1, 'face': 2, 'human': 3, 'carplate': 4`

Поменяем свой формат меток на необходимый

In [ ]:
my_to_site = {
    0: 3,
    1: 1,
    2: 2,
    3: 0,
    4: 4
}

solution_site_yolov7_df['LabelName'] = solution_site_yolov7_df.LabelName.apply(lambda x: my_to_site[x])

In [ ]:
!mkdir solutions

In [ ]:
solution_site_yolov7_df.to_csv("solutions/yolov7x_832_general_e163_conf01.csv",
                               sep=';', index=False)

## Проверка предсказания

Удостоверимся что файл сохранился

In [ ]:
path = "./solutions/yolov7x_832_general_e163_conf01.csv"

df = pd.read_csv(path, sep=';')
df.head()